<div class="alert alert-block alert-success">

# **1.** Environment Setup

<div>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 26 02:27:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


## 1.1 Connect Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os

# Change to the directory where project is located
os.chdir('/content/drive/MyDrive/College/MSc/2nd Semester/Deep Learning/project')

# Verify that we changed the directory
print("Changed directory to:", os.getcwd())

Changed directory to: /content/drive/MyDrive/College/MSc/2nd Semester/Deep Learning/project


## 1.2 Import Libraries

In [5]:
# Google Colab
!pip install keras_cv

In [6]:
import pandas as pd
import zipfile
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import regularizers
from classes import *
from functions import *

In [7]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.metrics import AUC, F1Score, CategoricalAccuracy, TopKCategoricalAccuracy
from sklearn.metrics import f1_score, precision_score, recall_score
from keras import optimizers

## 1.3 Import Dataset

The amount of data we have is not supported by GitHub (where we have our project stored). The solution is: create a folder named data and allocate the rare_species file inside it. The gitignore file makes sure this folder is not used when we are pulling or pushing changes but everyone needs to have it on their machines locally. A random seed was used to ensure that the splits stay the same.

In [8]:
# Import Metadata
metadata_path = Path("../data/rare_species/metadata.csv")
df = pd.read_csv(metadata_path)
df.head()

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,75fd91cb-2881-41cd-88e6-de451e8b60e2,12853737,449393,animalia,mollusca,unionidae,mollusca_unionidae/12853737_449393_eol-full-si...
1,28c508bc-63ff-4e60-9c8f-1934367e1528,20969394,793083,animalia,chordata,geoemydidae,chordata_geoemydidae/20969394_793083_eol-full-...
2,00372441-588c-4af8-9665-29bee20822c0,28895411,319982,animalia,chordata,cryptobranchidae,chordata_cryptobranchidae/28895411_319982_eol-...
3,29cc6040-6af2-49ee-86ec-ab7d89793828,29658536,45510188,animalia,chordata,turdidae,chordata_turdidae/29658536_45510188_eol-full-s...
4,94004bff-3a33-4758-8125-bf72e6e57eab,21252576,7250886,animalia,chordata,indriidae,chordata_indriidae/21252576_7250886_eol-full-s...


In [9]:
df.shape # 11983 images

(11983, 7)

<div class="alert alert-block alert-success">

# **2.** Preprocessing

<div>

In [10]:
#Load the DataFrames from the .pkl files
with open("../data/train_df.pkl", "rb") as f:
     train_df = pickle.load(f)

with open("../data/valid_df.pkl", "rb") as f:
     val_df = pickle.load(f)

with open("../data/test_df.pkl", "rb") as f:
     test_df = pickle.load(f)

with open("family_encoder.pkl", "rb") as f:
     family_encoder = pickle.load(f)

with open("phylum_encoder.pkl", "rb") as f:
     phylum_encoder = pickle.load(f)

In [11]:
# identify the minority class
minority_class = train_df['family'].value_counts()[train_df['family'].value_counts() < 25].index
minority_class = minority_class.to_list()

In [12]:
batch_size = 32 ## the less the better because in each epoch the model sees N / batch_size images
image_size = (224, 224)

preprocess = Preprocessor_with_phylum(image_size=image_size, batch_size=batch_size)

In [13]:
# num_images = 16 ##
# rows, cols = 4, 4 ##

# plot_batch(train_ds, class_names=class_names, num_images=num_images, rows=rows, cols=cols)

<div class="alert alert-block alert-success">

# **3.** Models

<div>

## EfficientNet

### Base line 1 (without preprocessing and without regularization)

#### Set-up

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [ ]:
# Compile with metrics
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

In [ ]:
# Load datasets
train_ds_en_no_proc_no_reg, family_class_names, phylum_class_names = preprocess.load_img(
    train_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=True,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

val_ds_en_no_proc_no_reg, _, _ = preprocess.load_img(
    val_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=False,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

#### Run

In [ ]:
# Image input pipeline
image_input = Input(shape=(224, 224, 3), name="image_input")  # Input for RGB image
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=image_input)  # Pretrained EfficientNet without final dense layers

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)  # Convert 4D feature map to 2D vector (batch_size, 2048)

# Phylum input (one-hot or multi-class vector with 5 classes)
phylum_input = Input(shape=(5,), name="phylum_input")  # Input for phylum-level info

# Combine image and phylum features
combined = Concatenate()([x, phylum_input])  # Concatenate the two inputs: (batch_size, 2048 + 5)
combined = Dense(256, activation='relu')(combined)  # Fully connected layer
output = Dense(202, activation='softmax')(combined)  # Final classification layer (202 family classes)

# Define the model
model_en_no_proc_no_reg = Model(inputs=[image_input, phylum_input], outputs=output)

# Compile the model
model_en_no_proc_no_reg.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=metrics
)

# Print the model summary
# model.summary()

In [ ]:
# Initialize the experiment
experiment_en_no_proc_no_reg = Experiment(
    model=model_en_no_proc_no_reg,
    train_ds=train_ds_en_no_proc_no_reg,
    val_ds=val_ds_en_no_proc_no_reg,
    experiment_name="eff-net_with_phylum_no_proc_no_reg",
    resume=False,
    steps_per_epoch=263
)

# Run the experiment
history_en_no_proc_no_reg = experiment_en_no_proc_no_reg.run_experiment(callbacks=None, epochs=25)

Epoch 1/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 251s 573ms/step - accuracy: 0.0784 - auc: 0.6456 - f1_macro: 0.0171 - f1_weighted: 0.0509 - loss: 5.0572 - top5_accuracy: 0.1751 - val_accuracy: 0.2209 - val_auc: 0.8292 - val_f1_macro: 0.0566 - val_f1_weighted: 0.1354 - val_loss: 4.2321 - val_top5_accuracy: 0.4302
Epoch 2/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 73s 279ms/step - accuracy: 0.2572 - auc: 0.8644 - f1_macro: 0.0765 - f1_weighted: 0.1690 - loss: 3.9565 - top5_accuracy: 0.4763 - val_accuracy: 0.3239 - val_auc: 0.8992 - val_f1_macro: 0.1160 - val_f1_weighted: 0.2217 - val_loss: 3.4231 - val_top5_accuracy: 0.5765
Epoch 3/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 272ms/step - accuracy: 0.3573 - auc: 0.9292 - f1_macro: 0.1519 - f1_weighted: 0.2662 - loss: 3.1592 - top5_accuracy: 0.6351 - val_accuracy: 0.4062 - val_auc: 0.9401 - val_f1_macro: 0.2043 - val_f1_weighted: 0.3214 - val_loss: 2.8550 - val_top5_accuracy: 0.6656
Epoch 4/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 70s 266ms/step - accuracy: 0.4559 - auc: 0.9

In [ ]:
# Load the experiment log
df = pd.read_csv('experiment_log.csv')

# Identify the latest experiment
max_id = df['id'].max()

# Filter the DataFrame to get the latest experiment
df_latest_experiment = df[df['id'] == max_id]

# Save the latest experiment log to a CSV file
df_latest_experiment.to_csv('phylum_models_results/efficient_net_phylum_1_no_proc_no_reg_history.csv', index=False)

df_latest_experiment

,id,experiment_name,epoch,train_accuracy,val_accuracy,train_loss,val_loss,f1_train_macro,f1_val_macro,f1_train_weighted,f1_val_weighted,top5_train_accuracy,top5_val_accuracy,timestamp
365,15,eff-net_with_phylum_no_proc_no_reg,1,0.1368,0.2209,4.7950,4.2321,0.0312,0.0566,0.0855,0.1354,0.2776,0.4302,2025-04-25 17:23:10
366,15,eff-net_with_phylum_no_proc_no_reg,2,0.2810,0.3239,3.7621,3.4231,0.0948,0.1160,0.1914,0.2217,0.5108,0.5765,2025-04-25 17:24:24
367,15,eff-net_with_phylum_no_proc_no_reg,3,0.3770,0.4062,3.0345,2.8550,0.1728,0.2043,0.2890,0.3214,0.6535,0.6656,2025-04-25 17:25:35
368,15,eff-net_with_phylum_no_proc_no_reg,4,0.4690,0.4769,2.4912,2.4328,0.2876,0.2900,0.3974,0.4059,0.7470,0.7407,2025-04-25 17:26:45
369,15,eff-net_with_phylum_no_proc_no_reg,5,0.5427,0.5248,2.0781,2.1235,0.3900,0.3642,0.4865,0.4615,0.8189,0.7869,2025-04-25 17:27:55
370,15,eff-net_with_phylum_no_proc_no_reg,6,0.6067,0.5559,1.7702,1.8960,0.5009,0.4231,0.5675,0.5068,0.8634,0.8292,2025-04-25 17:29:04
371,15,eff-net_with_phylum_no_proc_no_reg,7,0.6611,0.5876,1.5308,1.7336,0.5934,0.4810,0.6355,0.5499,0.8952,0.8459,2025-04-25 17:30:15


### Base line 2 (without preprocessing and with regularization)

#### Set-up

In [14]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [15]:
# Compile with metrics
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

In [16]:
# Load datasets
train_ds_en_no_proc_reg, family_class_names, phylum_class_names = preprocess.load_img(
    train_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=True,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

val_ds_en_no_proc_reg, _, _ = preprocess.load_img(
    val_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=False,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

#### Run

In [17]:
# Image input pipeline
image_input = Input(shape=(224, 224, 3), name="image_input")  # Input for RGB image
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=image_input)  # Pretrained EfficientNet without final dense layers

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)  # Convert 4D feature map to 2D vector (batch_size, 2048)

# Phylum input (one-hot or multi-class vector with 5 classes)
phylum_input = Input(shape=(5,), name="phylum_input")  # Input for phylum-level info

# Combine image and phylum features
combined = Concatenate()([x, phylum_input])  # Concatenate the two inputs: (batch_size, 2048 + 5)
combined = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(combined)  # regularization parameter
combined = layers.Dropout(0.5)(combined) # regularization layer
output = Dense(202, activation='softmax', kernel_regularizer=regularizers.l2(1e-4))(combined)  # Final classification layer (202 family classes)

# Define the model
model_en_no_proc_reg = Model(inputs=[image_input, phylum_input], outputs=output)

# Compile the model
model_en_no_proc_reg.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.01), # regularization parameter
    metrics=metrics
)

# Print the model summary
# model.summary()

In [18]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
]

In [ ]:
# Initialize the experiment
experiment_en_no_proc_reg = Experiment(
    model=model_en_no_proc_reg,
    train_ds=train_ds_en_no_proc_reg,
    val_ds=val_ds_en_no_proc_reg,
    experiment_name="eff-net_with_phylum_no_proc_reg",
    resume=True,
    steps_per_epoch=263,
)

# Run the experiment
history_en_no_proc_reg = experiment_en_no_proc_reg.run_experiment(callbacks=callbacks, epochs=50)

Resuming training from epoch 14 (timestamp 20250425-231537)
Epoch 15/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 189s 344ms/step - accuracy: 0.6226 - auc: 0.9826 - f1_macro: 0.5276 - f1_weighted: 0.5979 - loss: 1.7101 - top5_accuracy: 0.8769 - val_accuracy: 0.6060 - val_auc: 0.9775 - val_f1_macro: 0.5072 - val_f1_weighted: 0.5677 - val_loss: 1.7790 - val_top5_accuracy: 0.8598
Epoch 16/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 71s 271ms/step - accuracy: 0.6267 - auc: 0.9844 - f1_macro: 0.5363 - f1_weighted: 0.6039 - loss: 1.6691 - top5_accuracy: 0.8818 - val_accuracy: 0.6194 - val_auc: 0.9790 - val_f1_macro: 0.5340 - val_f1_weighted: 0.5850 - val_loss: 1.7449 - val_top5_accuracy: 0.8625
Epoch 17/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 70s 267ms/step - accuracy: 0.6429 - auc: 0.9862 - f1_macro: 0.5531 - f1_weighted: 0.6223 - loss: 1.6024 - top5_accuracy: 0.8927 - val_accuracy: 0.6288 - val_auc: 0.9793 - val_f1_macro: 0.5437 - val_f1_weighted: 0.5951 - val_loss: 1.7162 - val_top5_accuracy: 0.8648
Epoch 18/50
263/263 ━

### Set-up

In [14]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [15]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

initial_lr = 1e-4
final_lr = 1e-5
n_epochs = 50
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)
lr_callback = LearningRateScheduler(my_scheduler_fn)

In [16]:
# Load datasets
train_ds_en_pre_ft, family_class_names, phylum_class_names = preprocess.load_img(
    train_df,
    minority_class=minority_class,
    augment='mixup',
    oversampling=True,
    shuffle=True,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

val_ds_en_pre_ft, _, _ = preprocess.load_img(
    val_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=False,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

### Head Train

In [17]:
# Image input pipeline
image_input = Input(shape=(224, 224, 3), name="image_input")  # Input for RGB image
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=image_input)  # Pretrained EfficientNet without final dense layers

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)  # Convert 4D feature map to 2D vector (batch_size, 2048)

# Phylum input (one-hot or multi-class vector with 5 classes)
phylum_input = Input(shape=(5,), name="phylum_input")  # Input for phylum-level info

# Combine image and phylum features
combined = Concatenate()([x, phylum_input])  # Concatenate the two inputs: (batch_size, 2048 + 5)
combined = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(combined)  # regularization parameter
combined = layers.Dropout(0.5)(combined) # regularization layer
output = Dense(202, activation='softmax', kernel_regularizer=regularizers.l2(1e-4))(combined)  # Final classification layer (202 family classes)

# Define the model
model_en_pre_ft = Model(inputs=[image_input, phylum_input], outputs=output)

# Compile the model
model_en_pre_ft.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.01), # regularization parameter
    metrics=metrics
)

# Print the model summary
# model.summary()

In [18]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    lr_callback
]

In [ ]:
# Initialize the experiment
experiment_en_pre_ft = Experiment(
    model=model_en_pre_ft,
    train_ds=train_ds_en_pre_ft,
    val_ds=val_ds_en_pre_ft,
    experiment_name="eff-net_with_phylum_pre-ft",
    resume=True,
    steps_per_epoch=350,
)

# Run the experiment
history_en_pre_ft = experiment_en_pre_ft.run_experiment(callbacks=callbacks, epochs=50)

Resuming training from epoch 28 (timestamp 20250426-021318)
Epoch 29/50
350/350 ━━━━━━━━━━━━━━━━━━━━ 188s 257ms/step - accuracy: 0.7044 - auc: 0.9896 - f1_macro: 0.6684 - f1_weighted: 0.6979 - loss: 1.4654 - top5_accuracy: 0.9143 - val_accuracy: 0.6711 - val_auc: 0.9819 - val_f1_macro: 0.6225 - val_f1_weighted: 0.6520 - val_loss: 1.6119 - val_top5_accuracy: 0.8815 - learning_rate: 2.6303e-05
Epoch 30/50
350/350 ━━━━━━━━━━━━━━━━━━━━ 70s 200ms/step - accuracy: 0.6930 - auc: 0.9893 - f1_macro: 0.6584 - f1_weighted: 0.6870 - loss: 1.4813 - top5_accuracy: 0.9156 - val_accuracy: 0.6745 - val_auc: 0.9819 - val_f1_macro: 0.6298 - val_f1_weighted: 0.6565 - val_loss: 1.6084 - val_top5_accuracy: 0.8843 - learning_rate: 2.5119e-05
Epoch 31/50
350/350 ━━━━━━━━━━━━━━━━━━━━ 70s 202ms/step - accuracy: 0.7052 - auc: 0.9888 - f1_macro: 0.6710 - f1_weighted: 0.6999 - loss: 1.4603 - top5_accuracy: 0.9149 - val_accuracy: 0.6767 - val_auc: 0.9820 - val_f1_macro: 0.6334 - val_f1_weighted: 0.6585 - val_loss: 

In [20]:
# Load the experiment log
df = pd.read_csv('experiment_log.csv')

# Identify the latest experiment
max_id = df['id'].max()

# Filter the DataFrame to get the latest experiment
df_latest_experiment = df[df['id'] == max_id]

# Save the latest experiment log to a CSV file
#df_latest_experiment.to_csv('phylum_models_results/efficient_net_phylum_3_pre_ft_history.csv', index=False)

df_latest_experiment

,id,experiment_name,epoch,train_accuracy,val_accuracy,train_loss,val_loss,f1_train_macro,f1_val_macro,f1_train_weighted,f1_val_weighted,top5_train_accuracy,top5_val_accuracy,timestamp
400,18,eff-net_with_phylum_pre-ft,1,0.0672,0.2026,5.1371,4.6038,0.0261,0.0572,0.0490,0.1212,0.1567,0.3728,2025-04-26 00:13:07
401,18,eff-net_with_phylum_pre-ft,2,0.1718,0.2627,4.4422,3.8899,0.0696,0.0926,0.1050,0.1650,0.3427,0.5109,2025-04-26 00:14:19
402,18,eff-net_with_phylum_pre-ft,3,0.2352,0.3306,3.8712,3.3727,0.1263,0.1675,0.1637,0.2395,0.4771,0.6388,2025-04-26 00:15:30
403,18,eff-net_with_phylum_pre-ft,4,0.2933,0.3934,3.4293,2.9754,0.1982,0.2441,0.2310,0.3128,0.5908,0.7201,2025-04-26 00:16:45
404,18,eff-net_with_phylum_pre-ft,5,0.3582,0.4591,3.0522,2.6714,0.2791,0.3383,0.3077,0.3946,0.6635,0.7702,2025-04-26 00:26:13
405,18,eff-net_with_phylum_pre-ft,6,0.4065,0.5019,2.7715,2.4426,0.3478,0.3976,0.3684,0.4444,0.7187,0.7963,2025-04-26 00:27:25
406,18,eff-net_with_phylum_pre-ft,7,0.4481,0.5337,2.5524,2.2706,0.4034,0.4404,0.4191,0.4830,0.7552,0.8152,2025-04-26 00:28:36
407,18,eff-net_with_phylum_pre-ft,8,0.4816,0.5548,2.3701,2.1388,0.4432,0.4734,0.4558,0.5108,0.7865,0.8253,2025-04-26 00:29:51
408,18,eff-net_with_phylum_pre-ft,9,0.5068,0.5743,2.2208,2.0428,0.4719,0.5005,0.4837,0.5333,0.8090,0.8325,2025-04-26 00:36:55
409,18,eff-net_with_phylum_pre-ft,10,0.5304,0.5871,2.1190,1.9668,0.5054,0.5188,0.5129,0.5517,0.8233,0.8381,2025-04-26 00:38:07


In [ ]:
# acc_train_en = history_en.history['acc']
# acc_val_en = history_en.history['val_acc']
# plot_model_acc(num_epochs=50, train_acc=acc_train_en, val_acc=acc_val_en)

In [ ]:
# loss_train_en = history_en.history['loss']
# loss_val_en = history_en.history['val_loss']
# plot_model_loss(num_epochs=50, train_loss=loss_train_en, val_loss=loss_val_en)

In [ ]:
# x_axis = range(1,50+1)
# plt.plot(x_axis, history_en.history['f1_score'], 'r', label='Training F1-Score')
# plt.plot(x_axis, history_en.history['val_f1_score'], 'g', label='Validation F1-Score')
# plt.title('Training and Validation F1-Score')
# plt.xlabel('Epochs')
# plt.ylabel('F1-Score')
# plt.legend()
# plt.show()

### Fine-tune

References: https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=pt-br

"When fine-tuning a pre-trained model that contains BatchNormalization layers, it is usually a good idea to keep them frozen (i.e., set layer.trainable = False), to avoid corrupting the running statistics that the layers have learned."

In [ ]:
model_en_ft = load_model("eff-net_with_phylum_pre-ft_20250425-121418.keras", compile=False)

In [ ]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

initial_lr = 1e-5
final_lr = 1e-6
n_epochs = 100
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)
lr_callback = LearningRateScheduler(my_scheduler_fn)

In [ ]:
progressive_unfreeze = ProgressiveUnfreeze(model_en_ft) # class define in classes file

model_en_ft.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-5),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
    metrics=metrics
)

In [ ]:
# Initialize the experiment
experiment_en_ft = Experiment(
    model=model_en_ft,
    train_ds=train_ds_en_pre_ft,
    val_ds=val_ds_en_pre_ft,
    experiment_name="eff-net_with_phylum_ft",
    resume=True,
    steps_per_epoch=263,
)

# Run the experiment
history_ft_en = experiment_en_ft.run_experiment(
    epochs=100,
    callbacks=[progressive_unfreeze, lr_callback]
)

# Run the experiment
history_en_ft = experiment_en_ft.run_experiment(callbacks=callbacks, epochs=100)

No checkpoint found, starting from scratch.
Epoch 1/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - acc: 0.7133 - auc: 0.9891 - f1_score: 0.6941 - loss: 1.4267Epoch 0: Layer now unfrozen 241 (dense_7)
350/350 ━━━━━━━━━━━━━━━━━━━━ 113s 306ms/step - acc: 0.7133 - auc: 0.9891 - f1_score: 0.6942 - loss: 1.4267 - val_acc: 0.6867 - val_auc: 0.9801 - val_f1_score: 0.6611 - val_loss: 1.5418 - learning_rate: 9.7724e-06
Epoch 2/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 104s 296ms/step - acc: 0.7269 - auc: 0.9894 - f1_score: 0.7065 - loss: 1.4045 - val_acc: 0.6878 - val_auc: 0.9801 - val_f1_score: 0.6624 - val_loss: 1.5405 - learning_rate: 9.5499e-06
Epoch 3/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - acc: 0.7130 - auc: 0.9893 - f1_score: 0.6895 - loss: 1.4263Epoch 2: Layer now unfrozen 240 (dropout_3)
350/350 ━━━━━━━━━━━━━━━━━━━━ 100s 286ms/step - acc: 0.7130 - auc: 0.9893 - f1_score: 0.6896 - loss: 1.4262 - val_acc: 0.6889 - val_auc: 0.9799 - val_f1_score: 0.6628 - val_loss: 1.5397 - learning_rate

In [ ]:
# Load the experiment log
df = pd.read_csv('experiment_log.csv')

# Identify the latest experiment
max_id = df['id'].max()### Set-up

# Filter the DataFrame to get the latest experiment
df_latest_experiment = df[df['id'] == max_id]

# Save the latest experiment log to a CSV file
# df_latest_experiment.to_csv('phylum_models_results/efficient_net_phylum_3_pre_ft_history.csv', index=False)

df_latest_experiment

,id,experiment_name,epoch,train_accuracy,val_accuracy,train_loss,val_loss,f1_train_macro,f1_val_macro,f1_train_weighted,f1_val_weighted,top5_train_accuracy,top5_val_accuracy,timestamp
338,13,eff-net_with_phylum_no_proc_reg,1,0.0758,0.1859,5.0903,4.6492,0.0179,0.0404,0.0517,NaN,0.1684,0.3500,2025-04-24 21:58:37
339,13,eff-net_with_phylum_no_proc_reg,2,0.1946,0.2677,4.3801,4.0056,0.0511,0.0790,0.1217,NaN,0.3587,0.4669,2025-04-24 21:59:26
340,13,eff-net_with_phylum_no_proc_reg,3,0.2561,0.3228,3.8348,3.5320,0.0835,0.1129,0.1752,NaN,0.4793,0.5654,2025-04-24 22:00:15
341,13,eff-net_with_phylum_no_proc_reg,4,0.3161,0.3767,3.4094,3.1654,0.1357,0.1753,0.2382,NaN,0.5741,0.6311,2025-04-24 22:01:02
342,13,eff-net_with_phylum_no_proc_reg,5,0.3619,0.4207,3.0739,2.8756,0.1827,0.2224,0.2909,NaN,0.6369,0.6795,2025-04-24 22:01:51
343,13,eff-net_with_phylum_no_proc_reg,6,0.4082,0.4569,2.8134,2.6403,0.2343,0.2659,0.3420,NaN,0.6782,0.7151,2025-04-24 22:02:38
344,13,eff-net_with_phylum_no_proc_reg,7,0.4390,0.4975,2.5903,2.4480,0.2774,0.3179,0.3793,NaN,0.7208,0.7446,2025-04-24 22:03:27
345,13,eff-net_with_phylum_no_proc_reg,8,0.4704,0.5198,2.4086,2.2943,0.3282,0.3452,0.4187,NaN,0.7577,0.7718,2025-04-24 22:04:14
346,13,eff-net_with_phylum_no_proc_reg,9,0.5005,0.5448,2.2588,2.1675,0.3767,0.3893,0.4555,NaN,0.7830,0.7974,2025-04-24 22:05:02
347,13,eff-net_with_phylum_no_proc_reg,10,0.5295,0.5559,2.1257,2.0673,0.4149,0.4139,0.4897,NaN,0.8089,0.8114,2025-04-24 22:05:49


### Metrics

In [ ]:
# get_metric(val_ds_en_pre_ft, "efficient_net_finetuned_final.keras")
# get_metric(val_ds_en_pre_ft,"efficient_net_pre_finetuning_with_label_smoothing_batch_sized_corrected_launching_trying_lr_scheduler.keras")
# get_metric(val_ds_en_no_proc_reg, "model_Efficient_net_baseline2_20250423-115426.keras")
# get_metric(val_ds_en_no_proc_no_reg, "model_Efficient_net_baseline1_20250423-104546.keras")
# # custom function to include the metrics per class, as well as the precision and recall for error analysis purposes


Evaluating model: efficient_net_finetuned_final.keras


/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 34 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-04-24 11:36:30.785988: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defi

              precision    recall  f1-score   support

           0     0.7391    0.9444    0.8293        18
           1     0.6429    0.6923    0.6667        13
           2     0.6579    0.8065    0.7246        31
           3     0.7143    0.5556    0.6250         9
           4     0.6364    0.7778    0.7000        18
           5     0.6000    0.6000    0.6000         5
           6     0.0000    0.0000    0.0000         4
           7     0.6667    0.4000    0.5000         5
           8     1.0000    0.1111    0.2000         9
           9     1.0000    0.3333    0.5000         9
          10     0.6364    0.7778    0.7000        27
          11     0.7500    0.6667    0.7059         9
          12     1.0000    0.5556    0.7143         9
          13     1.0000    1.0000    1.0000        22
          14     0.7143    0.5556    0.6250         9
          15     1.0000    0.5000    0.6667         4
          16     0.5909    0.5909    0.5909        22
          17     1.0000    

2025-04-24 11:37:04.922697: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: Und

              precision    recall  f1-score   support

           0     0.6957    0.8889    0.7805        18
           1     0.6429    0.6923    0.6667        13
           2     0.6579    0.8065    0.7246        31
           3     0.7143    0.5556    0.6250         9
           4     0.6364    0.7778    0.7000        18
           5     0.5000    0.4000    0.4444         5
           6     0.0000    0.0000    0.0000         4
           7     0.7500    0.6000    0.6667         5
           8     1.0000    0.1111    0.2000         9
           9     1.0000    0.2222    0.3636         9
          10     0.6250    0.7407    0.6780        27
          11     0.7500    0.6667    0.7059         9
          12     1.0000    0.5556    0.7143         9
          13     1.0000    1.0000    1.0000        22
          14     0.7143    0.5556    0.6250         9
          15     1.0000    0.5000    0.6667         4
          16     0.6087    0.6364    0.6222        22
          17     1.0000    

2025-04-24 11:37:26.793685: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: Und

              precision    recall  f1-score   support

           0     0.6296    0.9444    0.7556        18
           1     0.5556    0.7692    0.6452        13
           2     0.6341    0.8387    0.7222        31
           3     0.8000    0.4444    0.5714         9
           4     0.5769    0.8333    0.6818        18
           5     1.0000    0.2000    0.3333         5
           6     0.0000    0.0000    0.0000         4
           7     1.0000    0.6000    0.7500         5
           8     1.0000    0.1111    0.2000         9
           9     1.0000    0.3333    0.5000         9
          10     0.5676    0.7778    0.6562        27
          11     0.4000    0.6667    0.5000         9
          12     1.0000    0.5556    0.7143         9
          13     1.0000    0.9545    0.9767        22
          14     0.5556    0.5556    0.5556         9
          15     1.0000    0.2500    0.4000         4
          16     0.5926    0.7273    0.6531        22
          17     1.0000    

2025-04-24 11:37:50.245592: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: Und